In [138]:
import os
import pandas as pd

In [139]:
pd.options.display.float_format = '{:,.2f}'.format

# Imports

In [140]:
userhome = os.path.expanduser('~')

# When I work from home
userhome = userhome.replace('C:', 'D:')

In [141]:
infolder = os.path.join(userhome, r'OneDrive - Transport Systems Catapult\Modelling\Data\NTEM\csv')

In [142]:
# Create a dict with the Lookups
lookups = {}

for fn in os.listdir(infolder):
    if 'tblLookUp' in fn:
        base = os.path.splitext(fn)[0]
        lookup_name = base.split('tblLookUp')[1]
        
        input_fp = os.path.join(infolder, fn) 
        df = pd.read_csv(input_fp, index_col=0)
        if 'ShortDesc' in df.columns:
            df = df['ShortDesc']
        lookups[lookup_name] = df.squeeze().to_dict()
        

In [143]:
lookups.keys()

dict_keys(['Geo72', 'Mode', 'Purpose', 'TimePeriod', 'TripType'])

In [144]:
infilepath = os.path.join(infolder, 'TripEndsByDirection_London.csv')

In [145]:
df = pd.read_csv(infilepath, index_col=list(range(5)))

C:\Users\haris.ballis\.conda\envs\py3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Lookup (MSOA to LAD)

In [146]:
infolder = os.path.join(userhome, r'OneDrive - Transport Systems Catapult\Modelling\Data\GIS\Boundaries\Lookups\OA')

In [147]:
infilepath = os.path.join(infolder, 'OA11_LSOA11_MSOA11_LAD11_EW_LUv2.csv')

In [148]:
lu = pd.read_csv(infilepath)
lu = lu[lu.columns[3:]].drop_duplicates()
lu.set_index('MSOA11CD', inplace=True)
lookups['MSOA11CD'] = lu

## Cleaning

In [149]:
df.columns = df.columns.astype('int')
df.columns.name = 'Year'
df = df.stack()
df.name = 'TripEnds'

## Danger! Zone 9999 gives the sum of each group, make sure it's filtered out

In [150]:
df.head(2)

ZoneID  Purpose  Mode  TimePeriod  TripType  Year
1310    1        1     1           1         2011   876.93
                                             2016   949.43
Name: TripEnds, dtype: float64

In [151]:
# Filter for the base year / We can interpolate of course
df = df.xs(2026, level='Year')

### Convert codes to names

In [152]:
grpby_bys = [None]
grpby_lvls = ['ZoneID']

for idxname in df.index.names:
    if idxname in lookups:
        grpby_bys.append(lookups[idxname])
        grpby_lvls.append(idxname)

In [153]:
df = df.groupby(by=grpby_bys, level=grpby_lvls).mean()
df.head(2)

ZoneID  Purpose  Mode  TimePeriod  TripType
1310    HBEB     Bus   AM          A           98.10
                                   D          103.90
Name: TripEnds, dtype: float64

In [154]:
# Convert to categorical data
df = df.reset_index()
cat_cols = ['Purpose', 'Mode', 'TimePeriod', 'TripType']

for c in df:
    if c in cat_cols:
        df[c] = df[c].astype('category')

idx_cols = ['ZoneID'] + cat_cols 
df = df.set_index(idx_cols)

tripEnds = df.copy()
tripEnds.head(2)

TripEnds
ZoneID Purpose Mode TimePeriod TripType          
1310   HBEB    Bus  AM         A            98.10
                               D           103.90

filt_modes = ['Car', 'CarPass']
filt_timeperiods = ['AM', 'IP', 'PM', 'OP']
filt_triptypes = ['O', 'D']

df = dfg.loc[idxslc[:, :, filt_modes, filt_timeperiods, filt_triptypes], :]
df.head()

## Aggregate purposes

In [155]:
# Aggregate purposes
df = tripEnds.copy()
purposes = list(df.index.levels[1].values)
aggrs = ['HBO', 'HBO', 'HBO', 'HBO', 'HBO', 'HBO', 'HBO', 'HBW', 'NHB', 'NHB', 'NHB', 'NHB', 'NHB', 'NHB', 'NHB']
d_aggrs = dict(list(zip(purposes, aggrs)))
print(d_aggrs)

df_aggr_purposes = df.groupby([None, d_aggrs, None, None, None], level=list(df.index.names)).sum()
df_aggr_purposes.head(2)

{'HBEB': 'HBO', 'HBEdu': 'HBO', 'HBHol': 'HBO', 'HBPB': 'HBO', 'HBRec': 'HBO', 'HBShop': 'HBO', 'HBVis': 'HBO', 'HBW': 'HBW', 'NHBEB': 'NHB', 'NHBEdu': 'NHB', 'NHBHol': 'NHB', 'NHBPB': 'NHB', 'NHBRec': 'NHB', 'NHBShop': 'NHB', 'NHBW': 'NHB'}


TripEnds
ZoneID Purpose Mode TimePeriod TripType          
1310   HBO     Bus  AM         A         1,904.84
                               D         2,338.83

# Convert to Model Zone System

In [156]:
# Convert ZoneID to MSOA11CD
d_msoa = lookups['Geo72']['NTEM7ZoneCode']
df = df_aggr_purposes.copy()
df = df.groupby(by=[d_msoa, None, None, None, None], level=df.index.names).sum()
df.index.rename('MSOA11CD', level=0, inplace=True)

tripEnds_msoa = df.copy()
tripEnds_msoa.head(2)

TripEnds
MSOA11CD  Purpose Mode TimePeriod TripType          
E02000001 HBO     Bus  AM         A        11,114.40
                                  D        11,629.20

In [157]:
# Create the ModelZoneIDs. The zone system uses NTEM codes.
# Internal Greenwich area is defined by MSOA codes and External by LAD codes

lu = lookups['MSOA11CD']
zoneIDs = lu.reset_index().apply(lambda x: x['MSOA11CD'] if x['LAD11NM']=='Greenwich' else x['LAD11CD'], axis=1)
zoneIDs.name = 'ZoneID'

# Create the dictionary that maps MSOA11CD to the new Model Zone System
keys = lu.index.values
values = zoneIDs.values
d_modelZones = dict(zip(keys, values))

In [158]:
# Decided to use msoas as model zones
tripEnds_modelZones = tripEnds_msoa.copy()

# Decided to use msoas for all London
# Convert the Zoning System
grp_bys = [d_modelZones, None, None, None, None]
grp_lvls = list(tripEnds_msoa.index.names)

tripEnds_modelZones = tripEnds_msoa.groupby(by=grp_bys, level=grp_lvls).sum()
tripEnds_modelZones.index.rename('ModelZoneID', level=0, inplace=True)
tripEnds_modelZones.head(2)

## Analysis

### Filtering

In [159]:
idxslc = pd.IndexSlice

In [160]:
df = tripEnds_modelZones.copy()

In [161]:
df.head(2)

TripEnds
MSOA11CD  Purpose Mode TimePeriod TripType          
E02000001 HBO     Bus  AM         A        11,114.40
                                  D        11,629.20

In [162]:
# Keep Origins and Destinations
df = df.loc[idxslc[:, :, :, 'AM IP PM OP'.split(), 'O D'.split()], :]

In [163]:
grp_mode_keys = list(df.index.get_level_values(2).unique())
grp_mode_vals = 'Bus CarDriver CarPass Other Rail Walk'.split()

dict_grp_mode = dict(zip(grp_mode_keys, grp_mode_vals))
dict_grp_mode

{'Bus': 'Bus',
 'CarDriver': 'CarDriver',
 'CarPass': 'CarPass',
 'Cycle': 'Other',
 'Rail': 'Rail',
 'Walk': 'Walk'}

In [164]:
# Seems like a bug, cannot use dictionary when groupby level is Categorical
# df.groupby([None, None, dict_grp_mode, None, None], level=[0, 1, 2, 3, 4]).sum()

# Aggregate modes for consistency with NTS
df = df.reset_index()
df.Mode = df.Mode.map(dict_grp_mode)
df.set_index(list(df.columns[:-1]), inplace=True)
origsdests_purpose_mode_time = df.groupby(level=df.index.names).sum()

In [165]:
# Convert to daily data
origsdests_purpose_mode_time = origsdests_purpose_mode_time.div(5)

## Export

In [166]:
outfolder = r'../GravityModel/inputs'

In [167]:
outfilepath = os.path.join(outfolder, 'tripends_msoas_2026_purpose_mode_time.csv')
origsdests_purpose_mode_time.to_csv(outfilepath)

# Playground

In [168]:
df = origsdests_purpose_mode_time.squeeze()

In [169]:
df.groupby(level=['TimePeriod', 'Purpose', 'TripType']).sum().div(5)

TimePeriod  Purpose  TripType
AM          HBO      D            549,604.32
                     O            567,895.59
            HBW      D            413,669.69
                     O            401,406.61
            NHB      D             66,394.89
                     O             61,773.19
IP          HBO      D          1,117,113.00
                     O          1,107,391.00
            HBW      D            161,958.70
                     O            161,223.14
            NHB      D            276,386.32
                     O            274,086.40
OP          HBO      D            403,575.12
                     O            407,978.63
            HBW      D            142,220.72
                     O            144,385.09
            NHB      D             40,426.04
                     O             43,811.83
PM          HBO      D            517,524.29
                     O            506,349.25
            HBW      D            303,063.97
                     O   

In [218]:
df = origsdests_purpose_mode_time.copy()

In [175]:
df

MSOA11CD   Purpose  Mode       TimePeriod  TripType
E02000001  HBO      Bus        AM          D          2,325.84
                                           O            910.49
                               IP          D          4,570.67
                                           O          4,545.73
                               OP          D            923.13
                                           O          2,097.48
                               PM          D          1,120.59
                                           O          3,014.10
                    CarDriver  AM          D          2,723.34
                                           O            860.84
                               IP          D          5,144.07
                                           O          4,917.50
                               OP          D          2,216.70
                                           O          3,699.85
                               PM          D          2,120.56
   

In [173]:
df_hb = df.loc[idxslc[:,['HBW','HBO'],:]]

In [179]:
df_hb.groupby(level=['TimePeriod', 'TripType']).sum()

TimePeriod  TripType
AM          D          4,816,370.07
            O          4,846,510.99
IP          D          6,395,358.48
            O          6,343,070.70
OP          D          2,728,979.22
            O          2,761,818.56
PM          D          4,102,941.28
            O          4,080,280.43
Name: TripEnds, dtype: float64